In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats

### 데이터 불러오기, null 값 삭제, 분석컬럼 추출, 연속형 -> 범주형으로 데이터 타입 변경

In [2]:
df_JCN = pd.read_csv('../../../dataset/Join Collection NSC2_BND_M20.csv')
df_JCN.head(2)

,Unnamed: 0,ED_RC_TOT_AMT,MCARE_SUBJ_CD,OPRTN_YN,MCARE_RSLT_TYPE,EDC_SBA,BTH_YYYY
0,0,7570,1,0,5.0,3000,1921LE
1,1,18160,1,0,5.0,6000,1921LE


In [3]:
df_JCN = df_JCN.dropna()

In [4]:
df_JCN.isnull().sum()

Unnamed: 0         0
ED_RC_TOT_AMT      0
MCARE_SUBJ_CD      0
OPRTN_YN           0
MCARE_RSLT_TYPE    0
EDC_SBA            0
BTH_YYYY           0
dtype: int64

In [5]:
df_JCN_SKY = df_JCN[['ED_RC_TOT_AMT','MCARE_SUBJ_CD','OPRTN_YN','MCARE_RSLT_TYPE']]
df_JCN_SKY[:2]

,ED_RC_TOT_AMT,MCARE_SUBJ_CD,OPRTN_YN,MCARE_RSLT_TYPE
0,7570,1,0,5.0
1,18160,1,0,5.0


In [6]:
df_JCN_SKY.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4205 entries, 0 to 4628
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ED_RC_TOT_AMT    4205 non-null   int64  
 1   MCARE_SUBJ_CD    4205 non-null   int64  
 2   OPRTN_YN         4205 non-null   int64  
 3   MCARE_RSLT_TYPE  4205 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 164.3 KB


In [7]:
df_JCN_SKY[['MCARE_SUBJ_CD','OPRTN_YN','MCARE_RSLT_TYPE']] = df_JCN_SKY[['MCARE_SUBJ_CD','OPRTN_YN','MCARE_RSLT_TYPE']].astype(str)

C:\Users\01-09\AppData\Local\Temp\ipykernel_1236\2275312239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_JCN_SKY[['MCARE_SUBJ_CD','OPRTN_YN','MCARE_RSLT_TYPE']] = df_JCN_SKY[['MCARE_SUBJ_CD','OPRTN_YN','MCARE_RSLT_TYPE']].astype(str)


In [8]:
df_JCN_SKY.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4205 entries, 0 to 4628
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ED_RC_TOT_AMT    4205 non-null   int64 
 1   MCARE_SUBJ_CD    4205 non-null   object
 2   OPRTN_YN         4205 non-null   object
 3   MCARE_RSLT_TYPE  4205 non-null   object
dtypes: int64(1), object(3)
memory usage: 164.3+ KB


## - 목표변수(Y) : ED_RC_TOT_AMT(요양급여비용총액)
## - 설명변수 (X) : MCARE_SUBJ_CD(진료과목코드), OPRTN_YN(수술여부), MCARE_RSLT_TYPE(진료결과구분)

### 선정변수 : MCARE_SUBJ_CD / ED_RC_TOT_AMT (범주형 / 연속형)
 - 분석내용 : 진료과목 코드에 따라 요양급여비용총액에 영향이 있을까?
   - 귀무가설 : 진료과목 코드에 따라 요양급여비용총액에 차이가 없다.
   - 대립가설 : 진료과목 코드에 따라 요양급여비용총액에 차이가 있다.
   
 - 분석 결론 : P.value (0.0407) < 0.05 : 대립가설 참
   - 통계 결론 : KruskalResult(statistic=6.402935921932224, pvalue=0.0407024105519787) 으로 대립가설 참
   - 사실 결론 : 진료과목 코드에 따라 요양급여비용총액에 차이가 있다.

#### 정규성 확인
 - 연속형인지 아닌지 확인
 - 5천개이하(4205)이므로 shapiro test 사용
 - statistic < cv(pvalue) : 정규 , statistic > cv(pvalue) : 비정규

In [10]:
stats.shapiro(df_JCN_SKY['ED_RC_TOT_AMT'])

##  statistic > cv 임으로 비정규

ShapiroResult(statistic=0.06868833303451538, pvalue=0.0)

#### 비정규 분포
- MCARE_SUBJ_CD(진료과목코드)는 범주형 3집단 이상 (kruskal 사용)
- ED_RC_TOT_AMT(요양급여비용총액)이 가장 높은 top3만 비교
- EDA에서 ED_RC_TOT_AMT(요양급여비용총액)이 가장 긴 MCARE_SUBJ_CD(진료과목코드) top3 추출 (1(내과), 23(가정의학과), 5(정형외과))

In [11]:
# 필터링, 추출 
conditions = (df_JCN_SKY['MCARE_SUBJ_CD']=='1') | (df_JCN_SKY['MCARE_SUBJ_CD']=='23') |(df_JCN_SKY['MCARE_SUBJ_CD']=='5')
conditions   
df_diagnosis = df_JCN_SKY[conditions][['MCARE_SUBJ_CD','ED_RC_TOT_AMT']]     

In [12]:
df_diagnosis.isnull().sum()

MCARE_SUBJ_CD    0
ED_RC_TOT_AMT    0
dtype: int64

In [13]:
df_diagnosis[:2]

,MCARE_SUBJ_CD,ED_RC_TOT_AMT
0,1,7570
1,1,18160


In [14]:
diagnosis_list = df_diagnosis['MCARE_SUBJ_CD'].value_counts().index.to_list()
diagnosis_list

['1', '23', '5']

In [16]:
df_result = [] # df_result =list()  # 각각 변수로 받아야하니깐 결과값을 list로 받음 
for item in diagnosis_list :
    condition = (df_diagnosis['MCARE_SUBJ_CD'] == item )
    df_result.append(df_diagnosis[condition]) 

In [17]:
len(df_result)

3

In [18]:
df_result[0][:2]

,MCARE_SUBJ_CD,ED_RC_TOT_AMT
0,1,7570
1,1,18160


In [19]:
stats.kruskal(df_result[0]['ED_RC_TOT_AMT'],df_result[1]['ED_RC_TOT_AMT'],df_result[2]['ED_RC_TOT_AMT'])

KruskalResult(statistic=6.402935921932224, pvalue=0.0407024105519787)

### 선정변수 : MCARE_RSLT_TYPE / ED_RC_TOT_AMT (범주형 / 연속형)
 - 분석내용 : 진료결과에 따라 요양급여비용총액에 영향이 있을까?
   - 귀무가설 : 진료결과에 따라 요양급여비용총액에 차이가 없다.
   - 대립가설 : 진료결과에 따라 요양급여비용총액에 차이가 있다.
   
 - 분석 결론 : P.value (6.26251) > 0.05 : 대립가설 참
   - 통계 결론 : KruskalResult(statistic=129.70452394994325, pvalue=6.262511796518197e-28) 으로 귀무가설 참
   - 사실 결론 : 진료결과에 따라 요양급여비용총액에 차이가 없다.

In [24]:
# 필터링, 추출 
conditions = (df_JCN_SKY['MCARE_RSLT_TYPE']=='5.0') | (df_JCN_SKY['MCARE_RSLT_TYPE']=='1.0') | (df_JCN_SKY['MCARE_RSLT_TYPE']=='9.0') |(df_JCN_SKY['MCARE_RSLT_TYPE']=='2.0')
conditions   
df_diagresult = df_JCN_SKY[conditions][['MCARE_RSLT_TYPE','ED_RC_TOT_AMT']]

In [25]:
df_diagresult.isnull().sum()

MCARE_RSLT_TYPE    0
ED_RC_TOT_AMT      0
dtype: int64

In [26]:
df_diagresult[:2]

,MCARE_RSLT_TYPE,ED_RC_TOT_AMT
0,5.0,7570
1,5.0,18160


In [27]:
result_list = df_diagresult['MCARE_RSLT_TYPE'].value_counts().index.to_list()
result_list

['1.0', '5.0', '9.0', '2.0']

In [28]:
df_result2 = [] # df_result =list()  # 각각 변수로 받아야하니깐 결과값을 list로 받음 
for item in result_list :
    condition = (df_diagresult['MCARE_RSLT_TYPE'] == item )
    df_result2.append(df_diagresult[condition])

In [29]:
len(df_result2)

4

In [30]:
df_result2[0][:2]

,MCARE_RSLT_TYPE,ED_RC_TOT_AMT
3,1.0,14200
6,1.0,11930


In [31]:
stats.kruskal(df_result2[0]['ED_RC_TOT_AMT'],df_result2[1]['ED_RC_TOT_AMT'],df_result2[2]['ED_RC_TOT_AMT'],df_result2[3]['ED_RC_TOT_AMT'])

KruskalResult(statistic=129.70452394994325, pvalue=6.262511796518197e-28)

### 선정변수 : OPRTN_YN / ED_RC_TOT_AMT (범주형 / 연속형)
 - 분석내용 : 수술 여부에 따라 요양급여비용총액에 영향이 있을까?
   - 귀무가설 : 수술 여부에 따라 요양급여비용총액에 차이가 없다.
   - 대립가설 : 수술 여부에 따라 요양급여비용총액에 차이가 있다.
   
 - 분석 결론 : P.value (0.00271) < 0.05 : 대립가설 참
   - 통계 결론 : RanksumsResult(statistic=-2.998573127148773, pvalue=0.0027124705352598944) 으로 대립가설 참
   - 사실 결론 : 수술 여부에 따라 요양급여비용총액에 차이가 있다.

#### 비정규 분포
- 수술여부는 범주형 2집단 (ranksums 사용)

In [32]:
df_OPRTN_no = df_JCN_SKY[df_JCN_SKY['OPRTN_YN'] == '0']
df_OPRTN_no[:2]

,ED_RC_TOT_AMT,MCARE_SUBJ_CD,OPRTN_YN,MCARE_RSLT_TYPE
0,7570,1,0,5.0
1,18160,1,0,5.0


In [33]:
df_OPRTN_yes = df_JCN_SKY[df_JCN_SKY['OPRTN_YN'] == '9']
df_OPRTN_yes[:2]

,ED_RC_TOT_AMT,MCARE_SUBJ_CD,OPRTN_YN,MCARE_RSLT_TYPE
4335,313930,1,9,1.0
4579,8420,1,9,5.0


In [34]:
stats.ranksums(df_OPRTN_no['OPRTN_YN'], df_OPRTN_yes['OPRTN_YN'])

RanksumsResult(statistic=-2.998573127148773, pvalue=0.0027124705352598944)